In [1]:
import os, random, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import openpyxl
import shutil
from scipy.stats import mode

In [2]:
if os.path.isdir('./data/.ipynb_checkpoints'):
    shutil.rmtree('./data/.ipynb_checkpoints')
f_list = os.listdir('./data/')
f_list.sort()

production_file = [i for i, v in enumerate(f_list) if 'PRODUCTION' in v.split('_')]
sale_file = [i for i, v in enumerate(f_list) if 'Fcst' in v.split(' ')]
inven_file = [i for i, v in enumerate(f_list) if 'INVENTORY' in v.split('_')]
print(f_list)
print(production_file, '\n', sale_file, '\n', inven_file)

['3_DAIMLER_E61D_SCP_PRODUCTION_PLANNING_v0512.xlsx', '4_DAIMLER_E61D_SCP_PRODUCTION_RESULT_2018_10_2021_03_v2.xlsx', '5_Daimler_E61D_INVENTORY_210512_v3.xlsx', 'Daimler_MktFcstWaterfall(Mkt Fcst Waterfall)_201901_202003.xlsx', 'Daimler_MktFcstWaterfall(Mkt Fcst Waterfall)_201909_202112.xlsx', 'Daimler_MktFcstWaterfall(Mkt Fcst Waterfall)_202004_202103.xlsx']
[0, 1] 
 [3, 4, 5] 
 [2]


In [3]:
def dta_merge(t_tmp, t_title):
    #t_tmp = tmp1 ; t_title = tmp1.columns
    ver_col = np.where(t_tmp.columns == 'Version')[0][0]
    t_tmp1 = pd.DataFrame(t_tmp.iloc[:,ver_col].copy())
    t_tmp1 = pd.concat([t_tmp1,t_tmp.iloc[:, 14:]], axis=1)
    return t_tmp1
def year_week_cal(tmp_set_, col_name_, format_):
    tmp_cal_year = list(pd.to_datetime(tmp_set_[col_name_], format=format_).dt.isocalendar().year.values)
    tmp_cal_ww   = pd.to_datetime(tmp_set_[col_name_], format=format_).dt.isocalendar().week.values
    tmp_cal_ww1 = list(map(lambda x : str(tmp_cal_ww[x]) if divmod(tmp_cal_ww[x], 10)[0] > 0 else str(0)+str(tmp_cal_ww[x]), range(0, len(tmp_cal_year))))
    tmp_cal_date = list(map(lambda x : str(tmp_cal_year[x]) + tmp_cal_ww1[x], range(0, len(tmp_cal_year))))
    return tmp_cal_date

In [4]:
for i in range(0, len(inven_file)):
    tmp_file_raw = openpyxl.load_workbook('./data/'+f_list[inven_file[i]])
    sheet_name = '02. 일별 PSI 트렌드' #tmp_file_raw.sheetnames
    tmp_file_pd = pd.DataFrame(tmp_file_raw[sheet_name].values).copy()
    tmp_col_name1 = list(tmp_file_pd.iloc[0,0:])
    tmp_file = tmp_file_pd.iloc[1:,:].copy().reset_index(drop=True)
    tmp_file.columns = tmp_col_name1
    tmp_day = pd.to_datetime(tmp_file['연월일'], format='%Y년%m월%d일').dt.isocalendar().day
    tmp_file1 = tmp_file[(tmp_file['자재'].str.slice(0,9) == 'ACEN1060I') & (tmp_day == 7)].copy()
    inven_filter_ = [['3', 'C'], ['3','T'],['5','0']]
    for k in range(0, len(inven_filter_)):
        tmp_store1, tmp_store2 = inven_filter_[k][0], inven_filter_[k][1]
        tmp_file1_sub1 = tmp_file1[(tmp_file1['저장위치'].str.slice(0,1) == tmp_store1)&(tmp_file1['저장위치'].str.slice(3,4) == tmp_store2)].copy()
        tmp_week_ = year_week_cal(tmp_file1_sub1, '연월일', '%Y년%m월%d일')
        tmp_file1_sub1['Inven Week'] = tmp_week_
        tmp_file1_sub1['Date'] = np.datetime_as_string(pd.to_datetime(tmp_file1_sub1['연월일'], format='%Y년%m월%d일'), unit='D')
        tmp_file1_sub1 = tmp_file1_sub1[['Inven Week','Date','재고수량']]
        tmp_group = tmp_file1_sub1.groupby(['Inven Week','Date'])
        tmp_sum = pd.DataFrame(tmp_group.sum())
        tmp_sum.reset_index(inplace=True)    
        tmp_sum.to_csv('./data1/Inven_%sxx%s.csv'%(tmp_store1, tmp_store2), index=False)
        print('Save Complete : Inven_%sxx%s.csv'%(tmp_store1, tmp_store2))

Save Complete : Inven_3xxC.csv
Save Complete : Inven_3xxT.csv
Save Complete : Inven_5xx0.csv


In [5]:
for i in range(0, len(production_file)): #1):#
    tmp_file_raw = openpyxl.load_workbook('./data/'+f_list[production_file[i]])
    if 'RESULT' in f_list[production_file[i]].split('_'):   
        sheet_name = '02.Detail_생산계획 준수율'
        tmp_file_pd = pd.DataFrame(tmp_file_raw[sheet_name].values).copy()
        tmp_col_name1 = list(tmp_file_pd.iloc[5,0:])
        tmp_file = tmp_file_pd.iloc[6:, :].copy().reset_index(drop=True)
        tmp_file.columns = tmp_col_name1
        tmp_file1 = tmp_file[(tmp_file['Material'].str.slice(0,9)=='ACEN1060I') & (tmp_file['Nick Name'].str.slice(0,4)=='E61D')].copy().reset_index(drop=True)
        tmp_file1 = tmp_file1[['Plan Date', 'Plan Week', 'Basic Date', 'Result Qty']] #'Plan Date', 
        #tmp_file1['WW'] = pd.to_datetime(tmp_file1['Plan Date']).dt.isocalendar().week
        tmp_file1 = tmp_file1[['Plan Week', 'Basic Date', 'Result Qty']] #'Plan Date', 
        tmp_group = tmp_file1.groupby(['Plan Week', 'Basic Date'])
        tmp_g_sum = pd.DataFrame(tmp_group.sum())
        tmp_g_sum.reset_index(inplace=True)
        #tmp_g_cnt = pd.DataFrame(tmp_group.count())
        #tmp_g_cnt.reset_index(inplace=True)
        tmp_g = tmp_g_sum#pd.merge(tmp_g_sum, tmp_g_cnt, on=['Plan Week', 'WW'])
    else:        
        sheet_name = 'DynamicResult'
        tmp_file_pd = pd.DataFrame(tmp_file_raw[sheet_name].values).copy()
        tmp_col_name1 = list(tmp_file_pd.iloc[0,0:])
        tmp_file = tmp_file_pd.iloc[1:, :].copy().reset_index(drop=True)
        tmp_file.columns = tmp_col_name1
        tmp_file1 = tmp_file[['PLAN_START_YYYYMMDD', 'YYYYMMDD', 'QTY']].copy() #'Plan Date', 
        tmp_plan_date_= year_week_cal(tmp_file1, 'PLAN_START_YYYYMMDD', '%Y%m%d')
        tmp_date_= year_week_cal(tmp_file1, 'YYYYMMDD', '%Y%m%d')
        tmp_file1['WW'] = tmp_plan_date_
        tmp_file1['tWW']=tmp_date_
        tmp_file1['Date'] = np.datetime_as_string(pd.to_datetime(tmp_file['PLAN_START_YYYYMMDD'], format='%Y%m%d'), unit='D')
        tmp_file1 = tmp_file1[['WW', 'Date', 'tWW', 'QTY']] #'PLAN_START_YYYYMMDD', 'YYYYMMDD', 
        tmp_group = tmp_file1.groupby(['WW', 'Date', 'tWW'])
        tmp_g_sum = pd.DataFrame(tmp_group.sum())
        tmp_g_sum.reset_index(inplace=True)
        #tmp_g_cnt = pd.DataFrame(tmp_group.count())
        #tmp_g_cnt.reset_index(inplace=True)
        tmp_g = tmp_g_sum #pd.merge(tmp_g_sum, tmp_g_cnt, on=['Plan Date', 'Date'])
    tmp_g.to_csv('./data1/'+sheet_name+".csv", index=False)

In [75]:
for f in range(0, len(sale_file)):    
    tmp_file_raw = openpyxl.load_workbook('./data/'+f_list[sale_file[f]])
    sheet_name = 'Mkt Fcst Waterfall'
    tmp_file = pd.DataFrame(tmp_file_raw[sheet_name].values)
    ind_col = np.where(tmp_file.iloc[0, :] == 'Measure')[0][0]
    col_name1 = list(tmp_file.iloc[0,:(ind_col+1)])
    tmp_col_name2 = list(tmp_file.iloc[0,(ind_col+1):])
    tmp_col_name2_ = []
    if None in tmp_col_name2:
        for i in range(0, len(tmp_col_name2)):    
            if tmp_col_name2[i] is not None:
                tmp_base = tmp_col_name2[i]
            tmp_col_name2_.append(tmp_base)
    else:
        tmp_col_name2_ = tmp_col_name2

    tmp_col_name3 = list(tmp_file.iloc[1,(ind_col+1):])
    col_name2 = []
    for i in range(0, len(tmp_col_name3)):
        if tmp_col_name3[i] == "SUM":
            col_name2.append(tmp_col_name3[i])
        else:
            if tmp_col_name2_[i][5:] == '12' and tmp_col_name3[i][1:] == '01':
                col_name2.append(str(int(tmp_col_name2_[i][:4])+1)+tmp_col_name3[i][1:])
            elif tmp_col_name2_[i][5:] == '01' and tmp_col_name3[i][1:] == '53':
                col_name2.append(str(int(tmp_col_name2_[i][:4])-1)+tmp_col_name3[i][1:])                
            else:
                col_name2.append(tmp_col_name2_[i][:4]+tmp_col_name3[i][1:])
    col_name = col_name1 + col_name2
    tmp_file1 = tmp_file.iloc[3:,:].reset_index(drop=True).copy()
    tmp_file1.columns = col_name
    if 'SUM' in col_name:
        rm_col1 = np.array([i for i, val in enumerate(col_name) if val == 'SUM'])
        tmp_file1.drop(tmp_file1.iloc[:, rm_col1], axis=1, inplace=True)
    keep_col_name = list(['Version']) + sorted(list(set(tmp_file1.columns[(ind_col+1):])))
    tmp_file1 = tmp_file1[keep_col_name].copy()
    tmp_file1_title = tmp_file1.columns.unique()
    tmp_file2_val = tmp_file1['Version'].copy()
    for k in range(1, len(tmp_file1_title)):
        tmp_i = [i for i, val in enumerate(tmp_file1.columns) if val == tmp_file1_title[k] ]
        #print(k, tmp_i, tmp_file1_title[k])
        if len(tmp_i) > 1:
            #tmp_file2.append(list(())
            tmp_file2_val = pd.concat([tmp_file2_val, tmp_file1.iloc[:,tmp_i].fillna(0).sum(axis=1)], axis=1)
        else :
            tmp_file2_val = pd.concat([tmp_file2_val, tmp_file1.iloc[:,tmp_i].fillna(0)], axis=1)
    tmp_file2_val.columns = tmp_file1_title
    if any(tmp_file2_val['Version'] == 'CURRENT'):
        tmp_file2_val = tmp_file2_val[tmp_file2_val['Version'] != 'CURRENT']    
    tmp_file2_val = tmp_file2_val.reset_index(drop=True)
    
    tmp_file2 = pd.DataFrame({'WW' : year_week_cal(tmp_file2_val, 'Version', 'MF_W_%Y%m%d_V001')})
    tmp_file2['Date'] = np.datetime_as_string(pd.to_datetime(tmp_file2_val['Version'], format='MF_W_%Y%m%d_V001'), unit='D')
    #list(map(lambda x: pd.to_datetime(tmp_file2_val['Version'], format='MF_W_%Y%m%d_V001').loc[x].strftime('%Y%m%d'), range(0, tmp_file2.shape[0])))
    tmp_file2 = pd.concat([tmp_file2, tmp_file2_val.drop(['Version'], axis=1)], axis=1)
    tmp_file2 = tmp_file2.sort_values(by=['Date'], axis=0)
    tmp_group = tmp_file2.groupby(['WW', 'Date'])
    tmp_g_sum = pd.DataFrame(tmp_group.sum())
    tmp_g_sum = tmp_g_sum.sort_index(axis=1)
    tmp_g_sum.reset_index(inplace=True)
    tmp_g_sum.to_csv('./data1/'+f_list[sale_file[f]]+'.csv', index=False)
    print(f_list[sale_file[f]])

Daimler_MktFcstWaterfall(Mkt Fcst Waterfall)_201901_202003.xlsx
Daimler_MktFcstWaterfall(Mkt Fcst Waterfall)_201909_202112.xlsx
Daimler_MktFcstWaterfall(Mkt Fcst Waterfall)_202004_202103.xlsx
